In [1]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import cv2
import numpy as np
from cvzone.HandTrackingModule import HandDetector
import tensorflow as tf

In [2]:
class GestureApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sign Gesture Recognition")
        self.root.configure(bg="#ADB2D4")
        
        # Initialize components
        self.model = tf.keras.models.load_model("keras_model.h5")
        self.detector = HandDetector(detectionCon=0.8, maxHands=1)
        self.cap = cv2.VideoCapture(0)
        self.img_references = []
        
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        self.create_widgets()
        self.update_video()


    def add_padding_and_resize(self, hand_img, size=(224, 224)):
        h, w = hand_img.shape[:2]
        aspect_ratio = w / h
    
        if aspect_ratio > 1:
            new_w = size[0]
            new_h = int(new_w / aspect_ratio)
        else:
            new_h = size[1]
            new_w = int(new_h * aspect_ratio)
        
        resized = cv2.resize(hand_img, (new_w, new_h))
        padded = np.zeros((size[1], size[0], 3), dtype=np.uint8)
        x_offset = (size[0] - new_w) // 2
        y_offset = (size[1] - new_h) // 2
        padded[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized
        gray = cv2.cvtColor(padded, cv2.COLOR_BGR2GRAY)
        return cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    
    def create_widgets(self):
       
        self.main_frame = tk.Frame(self.root, bg="#C7D9DD")
        self.main_frame.pack(pady=10, fill=tk.X)
        
        self.main_canvas = tk.Canvas(self.main_frame, width=600, height=400)
        self.main_canvas.pack(padx=10, pady=10)
        
      
        self.processing_frame = tk.Frame(self.root, bg="#D5E5D5")
        self.processing_frame.pack(pady=10, fill=tk.X, expand=True)
    
      
        self.canvas_container = tk.Frame(self.processing_frame, bg="#D5E5D5")
        self.canvas_container.pack()
    
        self.view_canvases = []
        for i in range(2):
            canvas = tk.Canvas(self.canvas_container, width=224, height=224)
            canvas.grid(row=0, column=i, padx=20, pady=5)  # Increased horizontal padding
            self.view_canvases.append(canvas)
        
     
        self.control_frame = tk.Frame(self.root, bg="#EEF1DA")
        self.control_frame.pack(pady=10, fill=tk.X)
        
        self.prediction_label = tk.Label(self.control_frame, 
                                       text="Prediction: ", 
                                       font=("Arial", 16),
                                       bg="#EEF1DA")
        self.prediction_label.pack(padx=20, pady=10)

    def process_image(self, img):
        hands, _ = self.detector.findHands(img, draw=True)
        processed_views = []
        
        if hands:
            hand = hands[0]
            x, y, w, h = hand['bbox']
            
          
            hand_crop = img[max(0,y-20):y+h+20, max(0,x-20):x+w+20]
            processed_views.append(hand_crop)
            
          
            gray_padded = self.add_padding_and_resize(hand_crop)
            processed_views.append(gray_padded)
       
            processed_img = self.preprocess_for_model(hand_crop)
            prediction = self.model.predict(processed_img)
            class_idx = np.argmax(prediction)
            self.show_prediction(class_idx)
            
        else:
        
            blank = np.zeros((224, 224, 3), np.uint8)
            processed_views = [blank, blank]
            self.prediction_label.config(text="Prediction: None")
            
        return processed_views
        
    def preprocess_for_model(self, hand_crop):
        processed = self.add_padding_and_resize(hand_crop)
        processed = processed.astype(np.float32) / 255.0
        return np.expand_dims(processed, axis=0)
    
    def show_prediction(self, class_idx):
        classes = ["ok", "rock", "silent", "thanks"]
        prediction = classes[class_idx]
        self.prediction_label.config(text=f"Prediction: {prediction}")
    
    def update_video(self):
        self.img_references = []
        ret, frame = self.cap.read()
        
        if ret:
            processed_views = self.process_image(frame)
            
           
            main_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            main_img = ImageTk.PhotoImage(Image.fromarray(main_img))
            self.img_references.append(main_img)
            self.main_canvas.create_image(0, 0, anchor=tk.NW, image=main_img)
            
           
            titles = ["Raw Crop", "Processed Input"]
            for i, view in enumerate(processed_views):
                view_img = cv2.cvtColor(view, cv2.COLOR_BGR2RGB)
                view_img = ImageTk.PhotoImage(Image.fromarray(view_img))
                self.img_references.append(view_img)
                self.view_canvases[i].create_image(0, 0, anchor=tk.NW, image=view_img)
                self.view_canvases[i].create_text(10, 10, text=titles[i], 
                                                 anchor=tk.NW, fill="white")
        
        self.root.after(10, self.update_video)
    
    def on_close(self):
       
        self.cap.release()
        cv2.destroyAllWindows()
        self.root.destroy()

In [3]:
root = tk.Tk()
app = GestureApp(root)
root.mainloop()

1/1 [==============================] - 0s 49ms/step
